In [131]:
import re

from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import Select

import numpy as np
import pandas as pd
#installation guide for selenium
#https://medium.com/@shanyitan/how-to-install-selenium-and-run-it-successfully-via-jupyter-lab-c3f50d22a0d4

In [132]:
#gathering information function
def gather(browser):
    lst=[]
    try:
        x=1
        while True: 
            x_pather = "//*[@id='myBody']/table/tbody/tr/td/table/tbody/tr[5]/td/form/table/tbody/tr[3]/td/a["+str(x)+"]"
            main = browser.find_element_by_xpath(x_pather)
            x+=1
            lst.append(main)
    except:
        lst.pop()
    return lst

### Scraping the data with selenium
*TODO we need to make it run by page

*TODO testing with other maslulim

In [133]:
browser = webdriver.Chrome()
browser.get("https://shnaton.huji.ac.il/")



select = Select(browser.find_element_by_id('faculty'))
select.select_by_value('07') #Pass value as string
sleep(1)

select = Select(browser.find_element_by_id('hugim'))
select.select_by_value('0824')
sleep(1)

select = Select(browser.find_element_by_id('maslul'))
select.select_by_value('7411')
sleep(1)

select = Select(browser.find_element_by_id('toar'))
select.select_by_value('1')
sleep(1)

select = Select(browser.find_element_by_id('shana'))
select.select_by_value('0')
sleep(1)

browser.find_element_by_css_selector("input[type='radio'][id='full']").click()
sleep(0.2)

browser.find_element_by_css_selector("input[type='button'][value=' חפש ']").click()

#collecting information
up = browser.find_element_by_xpath('//*[@id="myBody"]/table/tbody/tr/td/table/tbody/tr[5]/td/form/table/tbody/tr[3]/td/table[3]/tbody/tr/td[3]/a[2]')
try:
    while True:#information gathering function here
        browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        sleep(1)
        lst=gather(browser)
        up.click()
except:
    print(lst)
    browser.close



[<selenium.webdriver.remote.webelement.WebElement (session="090f038daff4266d2d7c2e51e96b3742", element="9a127abf-cf88-4001-874a-9aeb3c7ca04a")>, <selenium.webdriver.remote.webelement.WebElement (session="090f038daff4266d2d7c2e51e96b3742", element="2873c416-5161-46f9-9caa-2a28c4d88fbf")>, <selenium.webdriver.remote.webelement.WebElement (session="090f038daff4266d2d7c2e51e96b3742", element="e3af044f-b802-482e-bcf5-a01338332cb2")>, <selenium.webdriver.remote.webelement.WebElement (session="090f038daff4266d2d7c2e51e96b3742", element="c7a57845-4d4e-4220-b857-fca65d5fa5ec")>, <selenium.webdriver.remote.webelement.WebElement (session="090f038daff4266d2d7c2e51e96b3742", element="86f8b476-045d-4e4d-9b4d-fc117567e6ed")>, <selenium.webdriver.remote.webelement.WebElement (session="090f038daff4266d2d7c2e51e96b3742", element="225070e5-48f7-436c-9734-27981dccfe78")>, <selenium.webdriver.remote.webelement.WebElement (session="090f038daff4266d2d7c2e51e96b3742", element="17a62a0f-1f03-43aa-b651-c25541db

In [134]:
scraped_data = [i.text for i in lst]

### Gathering the data from the scrpaed text
gathering the data to dictionaries with regex.
we might have some weak points here.
#TODO test with other maslulim.

#### infrastructure 
database = {id:["semester": א\ב\שנתי, "day": א\ב\ג\ד\ה\ו, "hours": "hh:mm-hh:mm",....,..]}

from outside to inside:
big dictionary keys are for course_id, the value is a list which every item in the list is a different hour of the class.

In [137]:
database = dict()
canceled_courses = []
for course in scraped_data:
    course_details = []
    info_by_day = dict()
    
    
    hours = (re.findall(r"([0-5][0-9]:[0-5][0-9]-[0-5][0-9]:[0-5][0-9])", course))
    course_id = (re.findall(r"\|\s{3}(\d{5})", course))
    day = (re.findall(r"\n(יום [אבגדהו])", course))
    semester = (re.findall(r"\n(שנתי|סמסטר [אבגדהו])", course))
    
    #checking for canceled courses
    if len(course_id) == 0 or len(hours) == 0:
        if len(hours) == 0:
            canceled_courses.append(course_id)
        pass
    else:
        for item in range(len(hours)):
            info_by_day["semester"] = semester[item] 
            info_by_day["day"] = day[item]
            info_by_day["hours"] = hours[item]
            course_details.append(info_by_day)
        
        database[course_id[0]] = course_details

canceled_courses = [x for x in canceled_courses if x != []]

In [1]:
# #helper for regex
# counter = 0
# for i in scraped_data:
#     print(counter)
#     print(i,"\n"+"\n"+"*******end of object********"+"\n"+"\n")
#     counter += 1

In [12]:
birds = ['owl', 'hen', 'duck', 'parrot']
for bird in birds:
    if (len_bird := len(bird)) > 4:
        print(bird)
    else:
        print(len_bird)
# A bird “parrot” consists of “6”, letters

3
3
4
parrot


In [9]:
string = "print(f'A bird “{bird}” consists of “{len_bird}”, letters')"
string = string.replace("“", "'")
string = string.replace("”", "'")
string

"print(f'A bird '{bird}' consists of '{len_bird}', letters')"